In [29]:
import pandas as pd
from google.cloud import storage
import os.path
import os
import gzip
import shutil

NROWS = None

client = storage.Client()
bucket = client.get_bucket('facebook-posts')

for blob in bucket.list_blobs():
    if not blob.name.lower().endswith('.gzip'):
        continue
    path = os.path.join(os.getcwd(), blob.name)
    if not os.path.exists(path):
        print('Downloading {0} to {1} ...'.format(blob.name, path))
        with open(path, 'wb') as file_obj:
            blob.download_to_file(file_obj)
        print('Downloaded {0} to {1}!'.format(blob.name, path))
    
    print('Extracting {0}'.format(path))
    with gzip.open(path, 'rb') as f_in, open(path + ".csv", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
    print('Extracted {0}'.format(path))

Extracting /Users/Hoiy/project/tf_playground/posts-000000000000.gzip
Extracted /Users/Hoiy/project/tf_playground/posts-000000000000.gzip
Downloaded posts-000000000001.gzip to /Users/Hoiy/project/tf_playground/posts-000000000001.gzip!
Extracting /Users/Hoiy/project/tf_playground/posts-000000000001.gzip
Extracted /Users/Hoiy/project/tf_playground/posts-000000000001.gzip
Downloaded posts-000000000002.gzip to /Users/Hoiy/project/tf_playground/posts-000000000002.gzip!
Extracting /Users/Hoiy/project/tf_playground/posts-000000000002.gzip
Extracted /Users/Hoiy/project/tf_playground/posts-000000000002.gzip
Downloaded posts-000000000003.gzip to /Users/Hoiy/project/tf_playground/posts-000000000003.gzip!
Extracting /Users/Hoiy/project/tf_playground/posts-000000000003.gzip
Extracted /Users/Hoiy/project/tf_playground/posts-000000000003.gzip
Downloaded posts-000000000004.gzip to /Users/Hoiy/project/tf_playground/posts-000000000004.gzip!
Extracting /Users/Hoiy/project/tf_playground/posts-000000000004.

In [ ]:
posts = pd.DataFrame()
for i in range(40):
    print('./posts-0000000000{:0>2d}.gzip.csv'.format(i))
    posts = posts.append(pd.read_csv('./posts-{:0>12d}.gzip.csv'.format(i)), ignore_index=True)
    print(posts.shape)

./posts-000000000000.gzip.csv
(258109, 23)
./posts-000000000001.gzip.csv
(512727, 23)
./posts-000000000002.gzip.csv
(751558, 23)
./posts-000000000003.gzip.csv
(990036, 23)
./posts-000000000004.gzip.csv
(1285162, 23)
./posts-000000000005.gzip.csv
(1522273, 23)
./posts-000000000006.gzip.csv
(1786717, 23)
./posts-000000000007.gzip.csv
(2036815, 23)
./posts-000000000008.gzip.csv
(2286757, 23)
./posts-000000000009.gzip.csv


In [31]:
test = posts.head(10000)
test.shape

(10000, 23)

In [27]:
import numpy as np
test = posts[posts['status_type'] == 'published_story']
test['id']

245        330778070297798_924677370907862
285              184695207750_438694347750
1108      145300805513355_1038373536206073
1764        344460080577_10150309825765578
2075        136727474679_10150513586469680
2396         49249798825_10150428245218826
2496        110865425251_10150699141335252
2744       109062392492028_200658033332463
3306       146085438735912_996287687049012
3420         28198464871_10151051464034872
3776        193084362082_10150171780782083
4861       127432237332984_233378150071725
5407         83711079303_10153164981609304
5417          107322412447_101119429946997
5943        178550329805_10150394530129806
6374       117604218271416_180359785329192
6767         43394094549_10150289751959550
6856             139195819664_479614124664
7063       196264433742623_416691551699909
7620             171623547203_471102722203
7846          368278541524_119682861439780
8595       117604218271416_180646288633875
8924      146085438735912_1020302047980909
8933       

In [20]:
import TextPreprocessing.Language.detector as ld
pages['lang_name'] = pages['name'].apply(ld.detect)
pages['lang_description'] = pages['description'].astype(str).apply(ld.detect)
pages['lang_about'] = pages['about'].astype(str).apply(ld.detect)

#%timeit pages['name'].apply(lambda x: ld.detect(x, 'langdetect'))
#%timeit pages['about'].astype(str).apply(lambda x: ld.detect(x))
#%timeit pages['about'].astype(str).apply(lambda x: ld.detect(x, 'langdetect'))

In [27]:
eng_pages = pages[pages['lang_about'] == 'en']
print(eng_pages.shape)
eng_pages = eng_pages[eng_pages['lang_description'] == 'en']
print(eng_pages.shape)

(47230, 12)
(22790, 12)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import normal

x = np.linspace(0.0000001, 0.0002, 1000)
plt.hist(posts['share_ratio'], bins=x)
plt.show()

In [ ]:
x = np.linspace(0.0000001, 0.002, 1000)
plt.hist(posts['like_ratio'], bins=x)
plt.show()

In [ ]:
x = np.linspace(0.0000001, 0.0002, 1000)
plt.hist(posts['comment_ratio'], bins=x)
plt.show()

In [ ]:
link_posts = posts[posts['type'] == 'link']
link_posts.shape

In [ ]:
import langid

def isEng(x):
    try:
        return langid.classify(x['message'])[0] == 'en'
    except:
        try:
            return detect(x['description'])[0] == 'en'
        except:
            try:
                return detect(x['name'])[0] == 'en'
            except:
                return False
    

eng_link_posts = link_posts[link_posts.apply(isEng, axis=1)]

In [ ]:
eng_link_posts.shape
backup = eng_link_posts
data = eng_link_posts
eng_link_posts.iloc[0]['message']

In [ ]:
from langdetect import detect

def isEng(x):
    try:
        return detect(x['message']) == 'en'
    except:
        try:
            return detect(x['description']) == 'en'
        except:
            try:
                return detect(x['name']) == 'en'
            except:
                return False

eng_link_posts = link_posts[link_posts.apply(isEng, axis=1)]

In [ ]:
import csv
eng_link_posts.to_csv('/Users/Shared/fb/eng_link_posts.csv', doublequote=False, quotechar='"', escapechar='\\')

In [ ]:
test = pd.read_csv('/Users/Shared/fb/eng_link_posts.csv', nrows=10000, error_bad_lines=False, doublequote=False, quotechar='"', escapechar='\\')

In [ ]:
eng_link_posts[['name']].to_csv('/Users/Shared/fb/eng_link_posts_name.csv', index=False, header=False)

In [ ]:
eng_link_posts['data'] = eng_link_posts['name'].astype(str)
eng_link_posts['share_ratio'].describe()
eng_link_posts['good'] = eng_link_posts['share_ratio'].apply(lambda x: 1 if x >= 0.0001 else 0)
eng_link_posts['good'].describe()
data = eng_link_posts

In [ ]:
from codeDump.testHeadline import tokenize
tokenize(data['data'])
#test_data(data)

In [ ]:
# train word vector by fasttext

In [ ]:
data = eng_link_posts[['like', 'facebook_id', 'name', 'like_ratio']]
data['like_ratio'].describe()

In [ ]:
data['good'] = data['like_ratio'].apply(lambda x: 1 if x > 0.0005 else 0)
data['good'].describe()

In [ ]:
data["name"].describe()

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

data["name"] = data["name"].astype('str')

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data["name"])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
import os
import numpy as np

embeddings_index = {}
f = open(os.path.join('/Users/Shared/fb/eng_link_posts_name_model.vec'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
EMBEDDING_DIM = 100

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
def prepareData(df):    
    good = df[df["good"] == 1]
    bad = df[df["good"] == 0]
    bad = bad.sample(n=good.shape[0])
    data = good.append(bad)
    data = data.sample(frac=1).reset_index(drop=True)
    
    y_train = np.zeros((data.shape[0], 2), dtype=int)
    for i in range(0, data.shape[0]):
        y_train[i, 1] = 1 if data.iloc[i]['good'] == 1 else 0
        y_train[i, 0] = 1 if data.iloc[i]['good'] == 0 else 0
        
    sequences = tokenizer.texts_to_sequences(data["name"])
    x_train = pad_sequences(sequences, maxlen=25)

    return x_train, y_train, data['good'].values

In [ ]:
x_full, y_full, y2_full = prepareData(eng_link_posts)
x_train, y_train, y2_train = prepareData(train)
x_test, y_test, y2_test = prepareData(test)

print(y2_full)

In [ ]:
from sklearn.metrics import precision_score, recall_score

def validate(model, x_test, y_test):
    test_truth = np.apply_along_axis(lambda x: np.argmax(x), 1, y_test)
    test_pred = model.predict(x_test)
    test_pred = np.apply_along_axis(lambda x: np.argmax(x), 1, test_pred)
    precision = precision_score(test_truth, test_pred)
    recall = recall_score(test_truth, test_pred)
    print(precision)
    print(recall)
    return precision, recall

def validate_2(truth, pred):
    truth = np.apply_along_axis(lambda x: np.argmax(x), 1, truth)
    pred = np.apply_along_axis(lambda x: np.argmax(x), 1, pred)
    precision = precision_score(truth, pred)
    recall = recall_score(truth, pred)
    print(precision)
    print(recall)
    return precision, recall

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500, n_jobs=-1, criterion="entropy", random_state=1)
rf.fit(x_train, y2_train)

res = rf.predict(x_test)

#validate_2(y_test, res)
precision = precision_score(y2_test, res)
recall = recall_score(y2_test, res)
print(precision)
print(recall)

In [ ]:
origin = pd.read_csv('/Users/Shared/data/HN_posts_year_to_Sep_26_2016.csv')
origin_1 = origin[['id', 'num_points']]
data = pd.merge(title_vec, origin_1, how='inner', on=['id'])

In [ ]:
origin

In [ ]:
'''
sentence_length = pd.read_csv('sentence_length.csv', header=None, )
sentence_length.rename(columns={0: 'id', 1:'length'}, inplace=True)
data = pd.merge(data, sentence_length, how='inner', on=['id'])
'''

In [ ]:
GOOD_THRESHOLD = 10

data['good'] = data['num_points'].apply(lambda x: 1 if x >= GOOD_THRESHOLD else 0)
data['good'].describe()

In [ ]:
data = data.drop(['id', 'num_points'], 1)

In [ ]:
train = data.sample(frac=0.8)
test = data.drop(train.index)

abnormal = data.loc[data['good'] == 1]
abnormal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

m = train[list(range(0, 1200))].as_matrix()
u, s, v = np.linalg.svd(m, full_matrices=False)
plt.plot(s[0:400])
plt.show()

print(u.shape, s.shape, v.shape)
print(m[0].shape, v.transpose().shape)

transform = np.delete(v, range(2, v.shape[0]), 0)
print(transform.shape)

def reduce(vec):
    return np.dot(vec, transform.transpose())

def to_coor(matrix):
    coor = np.apply_along_axis(reduce, 1, matrix).transpose()
    print(matrix.shape, coor.shape)
    return (coor[0], coor[1])
    
normal_x, normal_y = to_coor(m)

m2 = abnormal[list(range(0,1200))].as_matrix()

ab_x, ab_y = to_coor(m2)

print(ab_x.shape)
print(ab_y.shape)

plt.scatter(normal_x, normal_y)
plt.scatter(ab_x, ab_y, color = 'red')
plt.show()

In [ ]:
m2 = abnormal.as_matrix()
u, s, v = np.linalg.svd(m2)
plt.plot(s[0:400])
plt.show()

In [ ]:
origin['num_points']

plot_data = origin[origin['num_points'] >= 0 ]['num_points'].values

x = np.array(list(range(100, 1001)))
y, x = np.histogram(plot_data, bins=x)
x_bin = np.delete(x, 0)

import matplotlib.pyplot as plt
plt.plot(x_bin, y)
plt.show()

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
def func(x, a, b):
    return a * np.exp(-b * x)

xdata = x_bin
ydata = y

print(xdata.shape)
print(ydata.shape)

popt, pcov = curve_fit(func, xdata, ydata, p0=[0.0007, 0.0004])

print(popt)
print(pcov)
plt.plot(x_bin, func(x_bin,*popt), color='g', linewidth=10)
plt.plot(x_bin, y, color='r')
plt.show()

from scipy.stats import expon
loc, lamb = expon.fit(plot_data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import normal

plt.hist(plot_data, bins=x_bin)
plt.plot(x_bin, func(x_bin,*popt), color='g', linewidth=3)
plt.show()

In [ ]:
a = np.array([1,2,3,4,5])
a[(a>1) & (a<5)]=0
a

np.log(1)

In [ ]:
#Study double / triple exponential behaviour of the distribution

x = np.linspace(0.0000001, 1, 100000)

def log_tower(plot_data, k):
    for i in range(0, k):
        plot_data[plot_data < 1] = 1.
        plot_data = np.log(plot_data)
    #plot_data[plot_data < 1] = 0.
    return plot_data

ln_plot_data = log_tower(np.array(plot_data), 3)

print(ln_plot_data.shape)

y, x = np.histogram(ln_plot_data, bins=x)
x_bin = np.delete(x, 0)

import matplotlib.pyplot as plt
plt.plot(x_bin, y)
plt.show()

In [ ]:
#Compute the elbow of exponential curve

x = np.linspace(0, 100, 10000)
y, x = np.histogram(plot_data, bins=x)
x_bin = np.delete(x, 0)

import matplotlib.pyplot as plt
plt.plot(x_bin, y)
plt.show()

In [ ]:
#1. Compute the derivative

def derivative(np_array):
    a = np.delete(np_array, len(np_array)-1)
    b = np.delete(np_array, 0)
    return a-b

d_y = derivative(derivative(y))
d_x_bin = np.linspace(0, len(d_y), len(d_y))

print(x_bin.shape)
print(d_y.shape)
print(d_x_bin.shape)

import matplotlib.pyplot as plt
plt.plot(d_x_bin, d_y)
plt.show()

print(np.argmax(d_y))

In [ ]:
# Find elbow from curve estimate

origin['num_points']

plot_data = origin[origin['num_points'] >= 0 ]['num_points'].values

x = np.linspace(0, 20, 20)
y, x = np.histogram(plot_data, bins=x)
x_bin = np.delete(x, 0)

import matplotlib.pyplot as plt
plt.plot(x_bin, y)
plt.show()

import numpy as np
from scipy.optimize import curve_fit
def func(x, a, b):
    return a * np.exp(-b * x)

xdata = x_bin
ydata = y

print(xdata.shape)
print(ydata.shape)

popt, pcov = curve_fit(func, xdata, ydata, p0=[0.0007, 0.0004])

print(popt)
print(pcov)
plt.plot(x_bin, func(x_bin,*popt), color='g', linewidth=10)
plt.plot(x_bin, y, color='r')
plt.show()

In [ ]:
a, b = popt
print(a)
print(b)

# second derivative of a * e^(-bx) is ab^2e^(-bx)

def firstD(a,b,x):
    return - a * b * np.exp(-b * x)


def secondD(a,b,x):
    return a * np.square(b) * np.exp(-b * x)

plt.plot(x_bin, secondD(a,b,x_bin))
plt.plot(x_bin, firstD(a,b,x_bin))
plt.show()

from keras.layers import Input, LSTM, RepeatVector, Dense
from keras.models import Model, Sequential

input_dim = 50
timesteps = 24
latent_dim = 200

predictor = Sequential()
predictor.add(LSTM(100, input_shape=(timesteps, input_dim)))
predictor.add(Dense(100, init='normal', activation='tanh'))
predictor.add(Dense(1, init='normal', activation='sigmoid'))

'''
decoded = RepeatVector(timesteps)(encoded)
#decoded = LSTM(input_dim, return_sequences=True)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)
'''

X = train.drop(['good'], axis=1)
Y = test.drop(['good'], axis=1)

X = X.as_matrix().reshape(train.shape[0], 24, 50)
Y = Y.as_matrix().reshape(test.shape[0], 24, 50)

predictor.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy', 'precision', 'recall', 'fmeasure'])
predictor.fit(X, train['good'], validation_data=(Y, test['good']))

sequence_autoencoder.evaluate(Y,test['good'])